In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 6.6 MB/s  eta 0:00:01


In [2]:
CLIENT_ID = 'FJSDO2FXNC3SRE033MXNHI45ZURYVXTM4NRYWAK1LDEQUQET' # your Foursquare ID
CLIENT_SECRET = 'NL2KOGL2TSTSR0I4G302FOCQNDD2SAF14RT1MPYEGBFB2KDI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FJSDO2FXNC3SRE033MXNHI45ZURYVXTM4NRYWAK1LDEQUQET
CLIENT_SECRET:NL2KOGL2TSTSR0I4G302FOCQNDD2SAF14RT1MPYEGBFB2KDI


In [3]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "New York City",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [4]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [5]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'New York', 'New York')

In [6]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 40.74841408113546, 'lng': -73.98373883019754},
  'sw': {'lat': 40.734344883713554, 'lng': -74.00235456585135}},
 174)

In [7]:
d["geocode"]

{'what': '',
 'where': 'new york city',
 'center': {'lat': 40.742185, 'lng': -73.992602},
 'displayString': 'New York, NY, United States',
 'cc': 'US',
 'geometry': {'bounds': {'ne': {'lat': 40.882214, 'lng': -73.907},
   'sw': {'lat': 40.679548, 'lng': -74.047285}}},
 'slug': 'new-york-city-new-york',
 'longId': '72057594043056517'}

In [8]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [9]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [10]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '49dbfd0ff964a520385f1fe3',
  'name': 'La Maison du Macaron',
  'location': {'address': '132 W 23rd St',
   'crossStreet': 'btwn 6th & 7th Ave',
   'lat': 40.74342660206714,
   'lng': -73.99399734674319,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.74342660206714,
     'lng': -73.99399734674319},
    {'label': 'entrance', 'lat': 40.743291, 'lng': -73.994105}],
   'postalCode': '10011',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['132 W 23rd St (btwn 6th & 7th Ave)',
    'New York, NY 10011',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d16a941735',
    'name': 'Bakery',
    'pluralName': 'Bakeries',
    'shortName': 'Bakery',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
     'suffix': '.png'},
    'p

In [11]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': '132 W 23rd St', 'crossStreet': 'btwn 6th & 7th Ave', 'lat': 40.74342660206714, 'lng': -73.99399734674319, 'labeledLatLngs': [{'label': 'display', 'lat': 40.74342660206714, 'lng': -73.99399734674319}, {'label': 'entrance', 'lat': 40.743291, 'lng': -73.994105}], 'postalCode': '10011', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['132 W 23rd St (btwn 6th & 7th Ave)', 'New York, NY 10011', 'United States']}
{'address': '22 W 25th St', 'crossStreet': 'btwn 6th Ave & Broadway', 'lat': 40.74362627711559, 'lng': -73.99021140882049, 'labeledLatLngs': [{'label': 'display', 'lat': 40.74362627711559, 'lng': -73.99021140882049}], 'postalCode': '10010', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['22 W 25th St (btwn 6th Ave & Broadway)', 'New York, NY 10010', 'United States']}
{'address': '124 W 25th St', 'crossStreet': 'btw 6th & 7th', 'lat': 40.74462646654926, 'lng': -73.992913428613

,uid,name,shortname,address,postalcode,lat,lng
0,49dbfd0ff964a520385f1fe3,La Maison du Macaron,Bakery,,10011,40.743427,-73.993997
1,5a5e29ae0868a24e8b9a8f1e,Maman,Café,,10010,40.743626,-73.990211
2,4a85d797f964a5206aff1fe3,Johny's Luncheonette,Diner,,10001,40.744626,-73.992913
3,54368cce498e7f27cef1d883,Dough,Donuts,,10011,40.739407,-73.992466
4,5890d8aecc5b6a43d6daa0f9,Gotham Coffee Roasters,Coffee Shop,,10011,40.739677,-73.992747


In [12]:
NewYorkCity_center = d["geocode"]["center"]
NewYorkCity_center

{'lat': 40.742185, 'lng': -73.992602}

In [15]:
from folium import plugins


map_NewYorkCity = folium.Map(location=[40.742185, -73.992602], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_NewYorkCity)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_NewYorkCity.add_child(plugins.HeatMap(hm_data))

map_NewYorkCity

In [17]:
lat = 40.742185 
lng = -73.992602
map_NewYorkCity = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Cafe Chill!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_NewYorkCity)
map_NewYorkCity